In [2]:
import pandas as pd
import numpy as np

In [19]:
train = pd.read_csv('data/reduce_train.csv')
new_col_names = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in train.columns]
train.columns = new_col_names

In [4]:
_target = 'accuracy_group'
_id = 'installation_id'

In [20]:
X_cols = [x for x in train.columns if x not in [_target, _id]]

In [21]:
X = train[X_cols]
y = train[_target]

In [7]:
from sklearn.model_selection import train_test_split

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [24]:
import lightgbm as lgb

In [25]:
params = {"objective" : "multiclass",
          "n_estimators": 50000,
          "num_class" : 4,
          "num_leaves" : 60,
          "max_depth": 6,
          "learning_rate" : 0.01,
          "bagging_fraction" : 0.9,  # subsample
          "feature_fraction" : 0.9,  # colsample_bytree
          "bagging_freq" : 5,        # subsample_freq
          "bagging_seed" : 42,
          "verbosity" : -1}

In [26]:
lgb_model = lgb.LGBMClassifier(**params)

In [28]:
lgb_model.fit(X_train,
              y_train,
              eval_set=[(X_test, y_test)],
              verbose=100,
              early_stopping_rounds=20
             )

Training until validation scores don't improve for 20 rounds
[40]	valid_0's multi_logloss: 1.14417
[80]	valid_0's multi_logloss: 1.09264
[120]	valid_0's multi_logloss: 1.06165
[160]	valid_0's multi_logloss: 1.0413
[200]	valid_0's multi_logloss: 1.02743
[240]	valid_0's multi_logloss: 1.01758
[280]	valid_0's multi_logloss: 1.01061
[320]	valid_0's multi_logloss: 1.00496
[360]	valid_0's multi_logloss: 1.00034
[400]	valid_0's multi_logloss: 0.997571
[440]	valid_0's multi_logloss: 0.995403
[480]	valid_0's multi_logloss: 0.993078
[520]	valid_0's multi_logloss: 0.991614
[560]	valid_0's multi_logloss: 0.99005
[600]	valid_0's multi_logloss: 0.988734
[640]	valid_0's multi_logloss: 0.987532
[680]	valid_0's multi_logloss: 0.987216
[720]	valid_0's multi_logloss: 0.986407
[760]	valid_0's multi_logloss: 0.985857
[800]	valid_0's multi_logloss: 0.985668
[840]	valid_0's multi_logloss: 0.985502
[880]	valid_0's multi_logloss: 0.985594
Early stopping, best iteration is:
[860]	valid_0's multi_logloss: 0.9854

LGBMClassifier(bagging_fraction=0.9, bagging_freq=5, bagging_seed=42,
               boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               feature_fraction=0.9, importance_type='split',
               learning_rate=0.01, max_depth=6, min_child_samples=20,
               min_child_weight=0.001, min_split_gain=0.0, n_estimators=50000,
               n_jobs=-1, num_class=4, num_leaves=60, objective='multiclass',
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0,
               verbosity=-1)

In [29]:
y_preds = lgb_model.predict(X_test)

In [30]:
from sklearn.metrics import confusion_matrix

In [31]:
confusion_matrix(y_test, y_preds)

array([[ 819,   61,    6,  372],
       [ 163,  109,   13,  452],
       [  77,   43,   16,  547],
       [ 164,   55,   15, 2395]], dtype=int64)

In [32]:
lgb_model.predict_proba(X_test)

array([[0.08467721, 0.09246488, 0.09029381, 0.7325641 ],
       [0.53891018, 0.09908131, 0.08052134, 0.28148717],
       [0.69066304, 0.09046516, 0.07877207, 0.14009973],
       ...,
       [0.05778397, 0.07632554, 0.07659405, 0.78929645],
       [0.95094789, 0.03337777, 0.007962  , 0.00771234],
       [0.05465113, 0.3003672 , 0.13582663, 0.50915504]])

In [34]:
print('Mejor score con todas las variables:', lgb_model.best_score_['valid_0']['multi_logloss'])

Mejor score con todas las variables: 0.9854644567757107


In [35]:
df_imp = pd.read_csv('importances/20200113_PermutationImportance_RF.csv')
df_imp.feature = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in df_imp.feature]

In [37]:
df_imp.feature = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in df_imp.feature]

In [38]:
df_imp.head()

,feature,Importance
0,session_title,0.021406
1,3393b68b,0.002261
2,accumulated_accuracy_group,0.001470
3,7525289a,0.001394
4,Chest_Sorter__Assessment__3121,0.001319
